# Big Data Project

## Configuration

In [ ]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [ ]:
val bucketname = "unibo-bd2122-dgrushchak"

val path = "s3a://"+bucketname+"/first-datasets/ml-movies.csv" //todo

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

In [ ]:
import java.util.Calendar
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner

object Parser {

  val noGenresListed = "(no genres listed)"
  val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
  val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
  val quotes = "\""
  
  //TODO check
  /** Convert from timestamp (String) to year (Int) */
  def yearFromTimestamp(timestamp: String): Int = {
    val cal = Calendar.getInstance()
    cal.setTimeInMillis(timestamp.trim.toLong * 1000L)
    cal.get(Calendar.YEAR)
  }


    // TODO redo
  /** Function to parse rating records
   *
   *  @param line line that has to be parsed
   *  @return tuple containing userId, movieId, rating, and year none in case of input errors
   */
  def parseRatingLine(line: String): Option[(Long, Long, Double, Int)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim.toLong, input(1).trim.toLong, input(2).trim.toDouble, yearFromTimestamp(input(3)))
    } catch {
      case _: Exception => None
    }
  }

}

In [ ]:
val rdd = sc.textFile(path).flatMap(Parser.parseRatingLine)

## Explorative queries

1. How many distinct airports and aircraft models
2. Average travel duration for airline
3. Percentage of basic economy tickets, based on all tickets
4. Percentage of non-stop flights (flights with one leg)
5. Average and price range of tickets
6. Average ticket price for each airline
7. Average and range of travel distance
8. Top 10 airports with more arriving flights flights


## Personal works

> Basta una query compicata a testa

Query approfondite:

- Denys:
    - Aggrego “airline” e “aircraft model” per determinare il prezzo medio di un miglio nautico, e dopo aggrego (MIN) su “aircraft model” per trovare l'airline che vende i biglietti più economici per quel modello.
    - Aggrego su “aircraft model” per calcolare la “travel distance” totale percorsa da ogni modello, poi faccio self-join e aggregazione per determinare il “travel duration” per ogni “aircraft model”. Alla fine determino la velocità di ogni modello partendo dai dati aggregati.
- Riccardo:
    - Aggrego su “aircraft models” per calcolare la classifica discreta dei modelli più usati rispetto a ciascuna “airline”. Eseguo il join col dataset originale, infine riaggrego su “travel duration” e sulla classificazione di prima. Ottengo la durata di media di ogni volo per ogni compagnia (oltre a numero di voli e totale di ore) sul modello di aereo più utilizzato
    - Aggrego su giorno ed airline per calcolare la distanza totale, poi aggrego su airline per calcolare la media dei totali giornalieri (di distanza) per ciascuna compagnia aerea
    - Aggrego su aeroporto di arrivo e di partenza, calcolando per ogni copia un AVG dei prezzi dei biglietti. Join con dataset originale. Aggrego nuovamente sulla classificazione di prima e sulla distanza totale percorsa. Ottengo così per ogni tratta il prezzo medio e la distanza media percorsa.

In [ ]:
val path_output = "s3a://"+bucketname+"/spark/avgRatPerMovie" //todo